# Préambule

## Quelques principes dans la collecte de données

La façon dont on récupère les données (scrapping) implique du bon sens dans la démarche afin d'éviter de surcharger les sites inutilement, ou de se faire considérer comme un robot, ce qui empècherait de collecter des données. Pour cela, on essaye au maximum de ne pas scrapper plusieurs fois des mêmes pages. Ainsi, on étudie à l'avance les variables d'intérêt et on teste à petite échelle si le code fonctionne avant de lancer le scrapping sur un nom de page plus grand.
De plus, on limite le nombre de requête temporel: on lance ainsi une requête tous les au plus 3 secondes. En pratique ce nombre est aléatoire entre 3 et 6 secondes ici pour éviter d'être détecter comme un automate.

## Philosophie du stockage des données

Le point précédent suppose que l'on stocke avec attention les données. On distingue pour cela deux fichiers: original_data.csv et working_data.csv. 
Le premier, original_data.csv, sert à stocker toutes les variables récupérées par scrapping, et ne sert qu'à cela. Il doit donc être sauvegardé fréquemment pour éviter une mauvaise manipulation qui aménerait à perdre les données.
Le second, working_data.csv, est le fichier de travail. Il s'agit initialement d'une copie de l'original, que l'on peut modifier pour obtenir de nouvelles variables d'intérêt, des données plus claires, etc. 

En théorie et idéalement, l'hérédité serait claire: working_data.csv serait "fils" de original_data.csv. En pratique, on peut avoir besoin de certains éléments de working_data.csv pour scrapper de nouveaux sites et rajouter des données à original_data.csv.

L'idée est également d'avoir une structure de données dynamique: on veut pouvoir ajouter des données à notre base, les traiter, puis en rajouter de nouvelles si besoin. Cela suppose que le code permette de rajouter simplement des données, et de faire des opérations (en particulier de scrapping) uniquement sur ces nouvelles données (souvent identifiables par des champs "NaN" dans certaines colonnes).

# Avant tout, quelques imports

On importe les packages de bibliothèques externe dont on a besoin

In [1]:
import time
import pandas as pd
import numpy as np
import random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

On importe les fonction de auxiliary.py dont on a besoin. Ce sont des fonctions "génériques" qui permettent par exemple de faire certaines opérations sur des dataframes.

In [2]:
from auxiliary_functions import remove_zero_start
from auxiliary_functions import intersect_list
from auxiliary_functions import complement_list
from auxiliary_functions import split_list

On importe les fonctions de scrapping dont on a besoin. Celle ci sont stockées dans un fichier Python à part pour éviter de surcharger ce Notebook.

In [3]:
from scrapping_function import add_expansion
from scrapping_function import add_tournament_use
from scrapping_function import add_price_trends

# Première étape de Scrapping: récupération de données de bases

On récupère des données de base sur les cartes de chaque extensions parmis une liste d'extension que l'on choisit.
On utilise pour cela le site de "magic card market", dans la section "Pokémon".
On récupère pour chaque carte d'une extension donnée:
- Le nom
- Le nom d'extension
- La date de sortie d'extension
- Le prix minimum auquel on peut trouver la carte
- Le nombre d'exemplaire en vente actuellement
- La rareté
- L'URL vers la page spécifique à la carte sur MKM, ou plus d'informations sont disponibles

Ces données, insuffisantes pour l'analyse, sont néanmoins le point de départ de recherches plus détaillées: les données de nom de carte (comportant un code d'identification) et de nom d'extension permettent d'aller chercher des informations plus précises facilement.

In [4]:
expansion_list =  ["Lost-Origin", "Stellar-Crown", "Surging-Sparks", "Shrouded-Fable", "Twilight-Masquerade", "Temporal-Forces", "Paldean-Fates", "Paradox-Rift", "151", "Obsidian-Flames", "Paldea-Evolved", "Scarlet-Violet", "Crown-Zenith"]

In [5]:
expansion_to_add = []
for expansion in expansion_to_add:
    add_expansion(expansion)

In [6]:
data = pd.read_csv("original_data.csv")
data.head(5)

,Index,Name,Expansion,Min price,Exemplaires en vente,Rareté,mkm_url,expansion_release_date,Tournament_last_month,Price trend,Price 7 days,Price 30 days
0,0,Live Code Card (Booster) (LOR),Lost-Origin,"0,02 €",14753,Online Code Card,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN
1,1,Thorton (LOR 167),Lost-Origin,"0,02 €",8496,Uncommon,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN
2,2,Rotom V (LOR 058),Lost-Origin,"0,70 €",239,Ultra Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",193.0,"3,69 €","3,56 €","3,23 €"
3,3,Colress's Experiment (LOR 155),Lost-Origin,"0,02 €",6339,Uncommon,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN
4,4,Lost Vacuum (LOR 162),Lost-Origin,"0,02 €",12538,Uncommon,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN


# Première sélection des données: barrière de prix

Le prix minimum des cartes est un bon indicateur nous permettant de faire un premier tri dans nos données. En effet, si notre variable d'intérêt est le prix de vente, il est inutile d'inclure toutes les cartes dont le prix est de 2 centimes, qui est le prix minimum de vente sur MKM: toutes ces cartes sont disponibles en abondance et n'ont pas de "rareté".
On enlève également certaines types de cartes: les cartes "Oversized" et "Online Code Card" qui ne sont pas jouables et ne seront pas étudiées dans notre cas.
On enlève enfin les cartes ayant des valeurs non acquise pour le prix (qui n'existent en réalité pas).

In [7]:
original_data = pd.read_csv("original_data.csv")
original_data.to_csv("working_data.csv", index = False)

data = pd.read_csv("working_data.csv")
data = data[(data["Rareté"] != "Online Code Card") & (data["Rareté"] != "Oversized")]
data = data.dropna(subset=["Min price"])

data["Min price"] = (data['Min price'].str[:-5] + data['Min price'].str[-4:-2]).astype(int)/100
data = data[(data["Min price"] >= 0.5)]

# Seconde étape de scrapping: récupération du nombre d'utilisation en tournoi

On récupère sur un autre site (limitlesstcg) le nombre d'utilisation de chaque carte en tournoi dans le dernier mois. 
Cette donnée est un facteur intéressant car les résultats en tournoi d'une carte peuvent inciter les gens à l'acheter, et donc faire varier le prix ou la disponibilité.

A partir de cette étape, on rajoute des données à notre tableau, mais pas nécessairement sur toutes les lignes. En effet, le jeu de donné étant volumineux, on ne peut scrapper individuellement les pages de chaque carte, et on choisit juste des catégories de cartes sur lesquels on veut plus d'information. En particulier, un moyen de sélectionner un ensemble de carte est la rareté.

On a également besoin de rajouter des colonnes dans notre tableau working_data pour scrapper plus facilement.

In [8]:
data[["Name", "Code"]] = data["Name"].str.split(" \(", expand = True)
data ["Code"] = data["Code"].str.rstrip(")")
data[["Expansion_code", "Number_code"]] = data["Code"].str.split(" ", expand = True)
data["Number_code"] = data["Number_code"].apply(remove_zero_start)
data.to_csv("working_data.csv", index = False)

On récupère toutes les lignes qui ont une rareté donnée et pour lesquelles on a pas encore récupéré la donnée du nombre de tournois joués le mois passé.

In [9]:
rarity = "Ultra Rare"
indexes = data.index[(data["Rareté"] == rarity) & pd.isna(data["Tournament_last_month"])].to_list()
indexes_list = split_list(indexes, step=15)

In [10]:
for indexes in indexes_list:
    add_tournament_use(indexes)

# Troisième étape de scrapping: on récupère des données plus précises sur les prix

En réalité, le prix minimum des cartes n'est pas un très bon indicateur du prix: cela peut traduire des cartes en mauvais états, ou des destinations mals desservies, résultant en des frais de port plus élevés.
Un meilleurs indicateur est présent sur la page de la carte: il s'agit du "Price Trend" qui est le prix moyen auquel s'échange la carte. En plus de cette variable, on récupère aussi le prix moyen sur les 7 et 30 derniers jours.

In [11]:
rarity = "Ultra Rare"
indexes = data.index[(data["Rareté"] == rarity) & pd.isna(data["Price trend"])].to_list()
indexes_list = split_list(indexes, step=15)

In [12]:
for indexes in indexes_list:
    add_price_trends(indexes)

# Etat des lieu des données et mise en forme

In [ ]:
original_data = pd.read_csv("original_data.csv")
original_data.to_csv("working_data.csv", index = False)

data = pd.read_csv("working_data.csv")

data = data[(data["Rareté"] != "Online Code Card") & (data["Rareté"] != "Oversized")]
data = data.dropna(subset=["Min price"])
data["Min price"] = (data['Min price'].str[:-5] + data['Min price'].str[-4:-2]).astype(int)/100
data = data[(data["Min price"] >= 0.5)]

data[["Name", "Code"]] = data["Name"].str.split(" \(", expand = True)

sub_data = data[data["Rareté"].isin(["Ultra Rare", "Secret Rare"])]
order = ["Index", "Name", "Expansion", "Min price", "Price trend", "Price 7 days", "Price 30 days", "Tournament_last_month"]
sub_data = sub_data[order]

In [26]:
len(sub_data)

230

In [25]:
sub_data.head()

,Index,Name,Expansion,Min price,Price trend,Price 7 days,Price 30 days,Tournament_last_month
2,2,Rotom V,Lost-Origin,0.7,"3,69 €","3,56 €","3,23 €",193.0
16,16,Giratina VSTAR,Lost-Origin,2.0,"3,20 €","3,33 €","3,29 €",131.0
80,80,Colress's Experiment,Lost-Origin,2.0,"3,57 €","2,63 €","3,31 €",70.0
101,101,Thorton,Lost-Origin,3.0,"4,56 €","4,80 €","4,60 €",223.0
114,114,Giratina V,Lost-Origin,4.9,"6,71 €","7,84 €","8,05 €",0.0
